<a href="https://colab.research.google.com/github/KeletsoM/JHB_Group_15_Sendy/blob/master/JHB_15_Sendy_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sendy Logistics report

## Table of Contents

## Background

## Objective

## Data cleaning

The raw data received from Sendy was has redundant and information that is not useful to predicting the Time from Pickup to Arrival. We therefore have to get rid of those particular data. Some of the data also needs to be transormed or converted to more useful information.

### Column Deletion
Firstly we need to get rid of useless and a redundant data the following is a list of all the columns that are going to be ommitted:

- <b>Order_ID</b>: this column carries no numerical or categorical significance, each entry is unique and has absolutely no bearing in predicting the Time from Pickup to Arrival.
- <b>User_ID</b>: the User_ID has no significance in our prediction because for our model we are assuming that Sendy does not discriminate or prioritise certain individuals.
- <b>Vehicle Types</b>: this column is going to be ommited because all the riders use a bike as their main form of transport.
- <b>Precipitation in millimeters</b>: this collumn has too much missing data 

In [0]:
import pandas as pd
Train = pd.read_csv('Train.csv')
Train.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,...,10:39:55 AM,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,11:16:16 AM,12,5,...,12:17:22 PM,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993
2,Order_No_1899,User_Id_265,Bike,3,Business,30,2,12:39:25 PM,30,2,...,1:00:38 PM,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455
3,Order_No_9336,User_Id_1402,Bike,3,Business,15,5,9:25:34 AM,15,5,...,10:05:27 AM,9,19.2,NaN,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341
4,Order_No_27883,User_Id_1737,Bike,1,Personal,13,1,9:55:18 AM,13,1,...,10:25:37 AM,9,15.4,NaN,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214


In [0]:
Train.drop(['Order No','User Id','Vehicle Type','Precipitation in millimeters'], axis=1, inplace=True)

In [0]:
Train

,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),...,Arrival at Destination - Weekday (Mo = 1),Arrival at Destination - Time,Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,3,Business,9,5,9:35:46 AM,9,5,9:40:10 AM,9,5,...,5,10:39:55 AM,4,20.4,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
1,3,Personal,12,5,11:16:16 AM,12,5,11:23:21 AM,12,5,...,5,12:17:22 PM,16,26.4,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993
2,3,Business,30,2,12:39:25 PM,30,2,12:42:44 PM,30,2,...,2,1:00:38 PM,3,NaN,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455
3,3,Business,15,5,9:25:34 AM,15,5,9:26:05 AM,15,5,...,5,10:05:27 AM,9,19.2,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341
4,1,Personal,13,1,9:55:18 AM,13,1,9:56:18 AM,13,1,...,1,10:25:37 AM,9,15.4,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21196,3,Personal,20,3,3:54:38 PM,20,3,3:55:09 PM,20,3,...,3,4:20:17 PM,3,28.6,-1.258414,36.804800,-1.275285,36.802702,Rider_Id_953,9
21197,3,Business,13,6,10:13:34 AM,13,6,10:13:41 AM,13,6,...,6,10:46:17 AM,7,26.0,-1.307143,36.825009,-1.331619,36.847976,Rider_Id_155,770
21198,3,Business,7,4,5:06:16 PM,7,4,5:07:09 PM,7,4,...,4,6:40:05 PM,20,29.2,-1.286018,36.897534,-1.258414,36.804800,Rider_Id_697,2953
21199,1,Personal,4,3,9:31:39 AM,4,3,9:31:53 AM,4,3,...,3,10:08:15 AM,13,15.0,-1.250030,36.874167,-1.279209,36.794872,Rider_Id_347,1380


From the table above we can observe that the all the deliveries that were provided in the dataset are all same day deliveries therefore having the following columns in our dataset would be redundant:

- Placement - Day of Month
- Placement - Weekday (Mo = 1)
- Confirmation - Day of Month
- Confirmation - Weekday (Mo = 1)
- Arrival at Pickup - Day of Month
- Arrival at Pickup - Weekday (Mo = 1)
- Arrival at Destination - Weekday (Mo = 1)

The approach is to keep 2 of the columns one for week day and for day of the month and delete the rest of the columns, but first we need to  confirm if they are equal.

In [0]:
Train.drop(['Confirmation - Day of Month','Confirmation - Weekday (Mo = 1)','Arrival at Pickup - Day of Month',
            'Arrival at Pickup - Weekday (Mo = 1)','Arrival at Destination - Weekday (Mo = 1)','Pickup - Day of Month',
            'Pickup - Weekday (Mo = 1)','Arrival at Destination - Day of Month'], axis=1, inplace=True)

### Data transformation

Data like the coordinates  for the pickup and the destination might look useless but they would be useful if they were transformed into names of cities and therefore we might get insights on how factors like trafic and navigation can impact the time from the pickup  to arrival. The process of converting coordinates into meaningful categorical data is refered to as reverse geocoding.

The reverse_geocoding library will be used to transform the data, before we can transform the coordinates we need to prepare the data by combining the the latitude and longitude values into tuples.

In [0]:
pickup_co= Train[['Pickup Lat', 'Pickup Long']].apply(tuple, axis=1).tolist()
destination_co = Train[['Destination Lat', 'Destination Long']].apply(tuple, axis=1).tolist()

Next we use the reverse_geocoding library to convert the coordinates into city names.

In [0]:
import reverse_geocoder as rg

pickup_re = rg.search(pickup_co, mode=2)
Train['Pickup_city'] = [x['name'] for x in pickup_re]

dest_re = rg.search(destination_co, mode=2)
Train['Destination_city'] = [x['name'] for x in dest_re]

In [0]:
Train.drop(['Pickup Lat','Pickup Long','Destination Lat','Destination Long'], axis=1, inplace=True)

In [0]:
Train.head()

,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Time,Arrival at Pickup - Time,Pickup - Time,Arrival at Destination - Time,Distance (KM),Temperature,Rider Id,Time from Pickup to Arrival,Pickup_city,Destination_city
0,3,Business,9,5,9:35:46 AM,9:40:10 AM,10:04:47 AM,10:27:30 AM,10:39:55 AM,4,20.4,Rider_Id_432,745,Nairobi,Nairobi
1,3,Personal,12,5,11:16:16 AM,11:23:21 AM,11:40:22 AM,11:44:09 AM,12:17:22 PM,16,26.4,Rider_Id_856,1993,Pumwani,Nairobi
2,3,Business,30,2,12:39:25 PM,12:42:44 PM,12:49:34 PM,12:53:03 PM,1:00:38 PM,3,NaN,Rider_Id_155,455,Pumwani,Nairobi
3,3,Business,15,5,9:25:34 AM,9:26:05 AM,9:37:56 AM,9:43:06 AM,10:05:27 AM,9,19.2,Rider_Id_855,1341,Nairobi,Nairobi
4,1,Personal,13,1,9:55:18 AM,9:56:18 AM,10:03:53 AM,10:05:23 AM,10:25:37 AM,9,15.4,Rider_Id_770,1214,Nairobi,Nairobi


### Handling missing Data